In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [ ]:
train = pd.read_csv("../input/train.tsv",sep = "\t")

In [ ]:
test = pd.read_csv("../input/test.tsv",sep = "\t")

In [ ]:
train

In [ ]:
test.head()

In [ ]:
sns.countplot(train["Sentiment"])

In [ ]:
train["Sentiment"].value_counts(normalize = True)*100

In [ ]:
train["number of words"] = train["Phrase"].apply(lambda x: len(x.split(" ")))
test["number of words"] = test["Phrase"].apply(lambda x: len(x.split(" ")))

In [ ]:
test[test.Phrase == " "]

In [ ]:
test.Phrase =  test.Phrase.replace(" ","no value")
train.Phrase =  train.Phrase.replace(" ","no value")


In [ ]:
sns.distplot(train["number of words"])
sns.distplot(test["number of words"])

In [ ]:
sns.violinplot("Sentiment","number of words",data = train)

In [ ]:
train["number of characters"] = train["Phrase"].apply(lambda x: sum(len(x) for x in x.split(" ")))
test["number of characters"] = test["Phrase"].apply(lambda x: sum(len(x) for x in x.split(" ")))

In [ ]:
sns.violinplot("Sentiment","number of characters",data = train)

In [ ]:
def avg_word_len(x):
    words = x.split(" ")
    avg_length = sum(len(word) for word in words)/len(words)
    return avg_length

In [ ]:
train["avg word length"] = train["Phrase"].apply(lambda x: avg_word_len(x))
test["avg word length"] = test["Phrase"].apply(lambda x: avg_word_len(x))

In [ ]:
sns.violinplot("Sentiment","avg word length",data = train)

In [ ]:
sns.distplot(train["avg word length"])
plt.show()
sns.distplot(test["avg word length"])
plt.show()

In [ ]:
#avg word length should be ommited ,giving distinction only after 25

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stop = stopwords.words("english")

In [ ]:
train["Phrase"] = train["Phrase"].apply(lambda x : x.lower())
test["Phrase"] = test["Phrase"].apply(lambda x : x.lower())

In [ ]:
def num_stop(x):
    words = x.split(" ")
    t = []
    for word in words:
        if word in stop:
            t.append(word)
    return len(t)

In [ ]:
train["number of stopwords"] = train["Phrase"].apply(lambda x : num_stop(x))
test["number of stopwords"] = test["Phrase"].apply(lambda x : num_stop(x))

In [ ]:
sns.violinplot("Sentiment","number of stopwords",data = train)

In [ ]:
sns.distplot(train["number of stopwords"])
plt.show()
sns.distplot(test["number of stopwords"])
plt.show()

In [ ]:
from nltk.stem import PorterStemmer
st = PorterStemmer()

In [ ]:
train["Phrase"] = train["Phrase"].apply(lambda x: " ".join([st.stem(word) for  word in x.split()]))
test["Phrase"] = test["Phrase"].apply(lambda x: " ".join([st.stem(word) for  word in x.split()]))

In [ ]:
train.head()

In [ ]:
train["Phrase"] = train["Phrase"].str.replace("[^\w\s]","")
test["Phrase"] = test["Phrase"].str.replace("[^\w\s]","")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(stop_words = "english")

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators = 200,max_depth = 100)


In [ ]:
data = tf.fit_transform(train.Phrase)

In [ ]:
xgb.fit(data,train.Sentiment)

In [ ]:
new = tf.transform(test.Phrase)

In [ ]:
test["Sentiment"] = xgb.predict(new)

In [ ]:
test[["PhraseId","Sentiment"]].to_csv("sample_submission.csv",index = False)